In [2]:
from langchain.llms import OpenAI
import os

In [3]:
os.environ['OPENAI_API_KEY'] = " "  # Add your key here

In [11]:
llm = OpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], temperature=0.6)

In [12]:
text = "what is the capital of France?"

In [13]:
print(llm.predict(text))

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

## Hugging face

In [19]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""  # add your key here

In [22]:
from langchain.llms import HuggingFaceHub

llm_huggingface = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature": 0.6, "max_length": 64})

In [16]:
output = llm_huggingface.predict("what is the capital of France?")

NameError: name 'llm_huggingface' is not defined

In [24]:
print(output)

paris


## Prompt templates and LLM chain

In [14]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(input_variables=["country"],
                                 template="Tell me the capital of this country - {country}")
prompt_template.format(country="India")

'Tell me the capital of this country - India'

In [15]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm_huggingface, prompt=prompt_template)
chain.invoke("india")

NameError: name 'llm_huggingface' is not defined

## Combining multiple chain using simple sequential chain

In [54]:
capital_prompt = PromptTemplate(input_variables=["country"], template="Please list the capital of the {country}")
capital_chain = LLMChain(llm=llm_huggingface, prompt=capital_prompt, )
famous_food_template = PromptTemplate(input_variables=["capital"],
                                      template="Suggest me a must try food from this {capital}")

In [55]:
famous_food_chain = LLMChain(llm=llm_huggingface, prompt=famous_food_template, )

In [56]:
from langchain.chains import SimpleSequentialChain

sim_seq_chain = SimpleSequentialChain(chains=[capital_chain, famous_food_chain])
sim_seq_chain.invoke(
    "France"
)

{'input': 'France', 'output': 'i would recommend the risotto'}

## Sequential chain

In [58]:
capital_prompt = PromptTemplate(input_variables=["country"], template="Please list the capital of the {country}")
capital_chain = LLMChain(llm=llm_huggingface, prompt=capital_prompt, output_key="capital")
famous_food_template = PromptTemplate(input_variables=["capital"],
                                      template="Suggest me a must try food from this {capital}")
famous_food_chain = LLMChain(llm=llm_huggingface, prompt=famous_food_template, output_key="food")

In [62]:
from langchain.chains import SequentialChain

Seq_chain = SequentialChain(chains=[capital_chain, famous_food_chain], input_variables=["country"],
                            output_variables=["capital", "food"])

In [63]:
Seq_chain({"country": "France"})

{'country': 'France',
 'capital': 'paris',
 'food': 'i would recommend the risotto'}

# Agents

In [4]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI
from dotenv import load_dotenv
import os

In [9]:
load_dotenv()
llm = OpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], temperature=0.6)
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

In [13]:
agent = initialize_agent(tools, llm, agent_type=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.invoke("When was the first world war? How long is it now in 2024 since that event ended?")



> Entering new AgentExecutor chain...
 I should use Wikipedia to find information about the first world war.
Action: wikipedia
Action Input: "first world war"
Observation: Page: World War I
Summary: World War I or the First World War (28 July 1914 – 11 November 1918) was a global conflict fought between two coalitions: the Allies and the Central Powers.  Fighting took place throughout Europe, the Middle East, Africa, the Pacific, and parts of Asia. One of the deadliest wars in history, it resulted in an estimated 9 million soldiers dead and 23 million wounded, plus another 5 million civilian deaths from various causes. Millions more died as a result of genocide, and the war was a major factor in the 1918 Spanish flu pandemic.
Increasing diplomatic tension between the European great powers reached a breaking point on 28 June 1914, when a Bosnian Serb named Gavrilo Princip assassinated Archduke Franz Ferdinand, heir to the Austro-Hungarian throne. Austria-Hungary held Serbia responsibl

{'input': 'When was the first world war? How long is it now in 2024 since that event ended?',
 'output': 'The first world war ended 106 years ago in 1918.'}

## Memory

In [17]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain import OpenAI

memory = ConversationBufferMemory()
prompt_template  = PromptTemplate(input_variables=["country"], template="Tell me the capital of this country - {country}")
llm_openai = OpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], temperature=0.55)

chain = LLMChain(llm=llm_openai, prompt=prompt_template)
chain.invoke("india")

{'country': 'india', 'text': '\n\nThe capital of India is New Delhi.'}

In [18]:
dir(chain) # thers memory attribute

['Config',
 'InputType',
 'OutputType',
 '__abstractmethods__',
 '__annotations__',
 '__call__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__orig_bases__',
 '__parameters__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__ror__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_updat

In [20]:
print(chain.memory) # nothing as of now

None


In [21]:
chain_mem = LLMChain(llm=llm_openai, prompt=prompt_template,memory=memory)

In [22]:
chain_mem.invoke("india")

{'country': 'india',
 'history': '',
 'text': '\n\nThe capital of India is New Delhi.'}

In [23]:
print(chain_mem.memory) # now we have the conversation stored

chat_memory=ChatMessageHistory(messages=[HumanMessage(content='india'), AIMessage(content='\n\nThe capital of India is New Delhi.')])


In [25]:
print(chain_mem.memory.buffer)

Human: india
AI: 

The capital of India is New Delhi.


In [27]:
# dont neeed to send all the memory to the openAI , this will be loads of tokens so well just restrict it to the last 5
from langchain.chains import ConversationChain
convo = ConversationChain(llm=llm_openai)
#chek the default template
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [28]:
convo.invoke("who won the 2014 FIFA world cup?")

{'input': 'who won the 2014 FIFA world cup?',
 'history': '',
 'response': " The 2014 FIFA World Cup was won by Germany. They beat Argentina with a score of 1-0 in the final match. It was held in Brazil and was the 20th edition of the tournament. Germany's Mario Götze scored the winning goal in the 113th minute of the match. This was Germany's fourth World Cup title and their first since 1990."}

In [30]:
convo.invoke("who won the 2018 FIFA world cup?")

{'input': 'who won the 2018 FIFA world cup?',
 'history': "Human: who won the 2014 FIFA world cup?\nAI:  The 2014 FIFA World Cup was won by Germany. They beat Argentina with a score of 1-0 in the final match. It was held in Brazil and was the 20th edition of the tournament. Germany's Mario Götze scored the winning goal in the 113th minute of the match. This was Germany's fourth World Cup title and their first since 1990.",
 'response': " The 2018 FIFA World Cup was won by France. They beat Croatia with a score of 4-2 in the final match. It was held in Russia and was the 21st edition of the tournament. France's Kylian Mbappé became the second teenager in history to score a goal in the World Cup final. This was France's second World Cup title, with their first being in 1998."}

In [31]:
convo.invoke("who was the winning captain of 2014 FIFA world cup?")

{'input': 'who was the winning captain of 2014 FIFA world cup?',
 'history': "Human: who won the 2014 FIFA world cup?\nAI:  The 2014 FIFA World Cup was won by Germany. They beat Argentina with a score of 1-0 in the final match. It was held in Brazil and was the 20th edition of the tournament. Germany's Mario Götze scored the winning goal in the 113th minute of the match. This was Germany's fourth World Cup title and their first since 1990.\nHuman: who won the 2018 FIFA world cup?\nAI:  The 2018 FIFA World Cup was won by France. They beat Croatia with a score of 4-2 in the final match. It was held in Russia and was the 21st edition of the tournament. France's Kylian Mbappé became the second teenager in history to score a goal in the World Cup final. This was France's second World Cup title, with their first being in 1998.",
 'response': ' The winning captain of the 2014 FIFA World Cup was Philipp Lahm. He was the captain of the German team and also played as a right-back. Lahm had been 

In [32]:
print(convo.memory.buffer)

Human: who won the 2014 FIFA world cup?
AI:  The 2014 FIFA World Cup was won by Germany. They beat Argentina with a score of 1-0 in the final match. It was held in Brazil and was the 20th edition of the tournament. Germany's Mario Götze scored the winning goal in the 113th minute of the match. This was Germany's fourth World Cup title and their first since 1990.
Human: who won the 2018 FIFA world cup?
AI:  The 2018 FIFA World Cup was won by France. They beat Croatia with a score of 4-2 in the final match. It was held in Russia and was the 21st edition of the tournament. France's Kylian Mbappé became the second teenager in history to score a goal in the World Cup final. This was France's second World Cup title, with their first being in 1998.
Human: who was the winning captain of 2014 FIFA world cup?
AI:  The winning captain of the 2014 FIFA World Cup was Philipp Lahm. He was the captain of the German team and also played as a right-back. Lahm had been the captain of the German national

In [39]:
from langchain.memory import ConversationBufferWindowMemory
window_memory = ConversationBufferWindowMemory(k=1)
convo = ConversationChain(llm=llm_openai,memory=window_memory)

In [40]:
convo.invoke("who won the 2018 FIFA world cup?")

{'input': 'who won the 2018 FIFA world cup?',
 'history': '',
 'response': " The 2018 FIFA World Cup was won by France, who defeated Croatia 4-2 in the final match. This was France's second World Cup victory, with their first being in 1998. The tournament was held in Russia and featured 32 teams from around the world. France's star player, Kylian Mbappé, became the youngest player since Pele to score in a World Cup final at the age of 19. The tournament also saw a record-breaking number of goals scored, with a total of 169 goals in 64 matches."}

In [41]:
convo.invoke("who was the captain then?")

{'input': 'who was the captain then?',
 'history': "Human: who won the 2018 FIFA world cup?\nAI:  The 2018 FIFA World Cup was won by France, who defeated Croatia 4-2 in the final match. This was France's second World Cup victory, with their first being in 1998. The tournament was held in Russia and featured 32 teams from around the world. France's star player, Kylian Mbappé, became the youngest player since Pele to score in a World Cup final at the age of 19. The tournament also saw a record-breaking number of goals scored, with a total of 169 goals in 64 matches.",
 'response': " The captain of the French team during the 2018 FIFA World Cup was Hugo Lloris. Lloris has been the captain of the French national team since 2012 and has made over 100 appearances for the team. He is also the captain of Tottenham Hotspur, a Premier League club in England. Lloris was a key player in France's victory, making crucial saves throughout the tournament."}

In [43]:
convo.invoke("what is 5+5?")

{'input': 'what is 5+5?',
 'history': "Human: who was the vice captain then?\nAI:  The vice captain of the French team during the 2018 FIFA World Cup was Raphaël Varane. Varane is a defender for Real Madrid and has been the vice captain of the French national team since 2017. He has also won multiple Champions League titles with Real Madrid and was a key player in France's defense during the World Cup.",
 'response': '  The answer to 5+5 is 10. This is a basic math equation that can be solved by adding two numbers together. In this case, both numbers are 5, so the answer is 10.'}

In [44]:
convo.invoke("who was the cental forward then?")

{'input': 'who was the cental forward then?',
 'history': 'Human: what is 5+5?\nAI:   The answer to 5+5 is 10. This is a basic math equation that can be solved by adding two numbers together. In this case, both numbers are 5, so the answer is 10.',
 'response': ' I am not sure what you are asking. Could you provide more context or clarify your question?'}